In [ ]:
import numpy as np
import pandas as pd

data = pd.read_csv('/content/sample_data/mnist_train_small.csv', header=None).to_numpy()

x = data[:, 1:]
y = data[:, 0]

print('x', x.shape)
print('y', y.shape)

x (20000, 784)
y (20000,)


In [ ]:
#Dataset, DataLoader
import torch
from torch.utils.data import Dataset, DataLoader

class SmallMNIST(Dataset):
  def __init__(self, x, y):
    self.x = x
    self.y = y

  def __len__(self):
    return self.x.shape[0]

  def __getitem__(self, idx):
    tx = torch.tensor(self.x[idx], dtype=torch.float32)
    ty = torch.tensor(self.y[idx], dtype=torch.long)
    return tx, ty

ds = SmallMNIST(x, y)
print('num samples:', len(ds))

loader = DataLoader(ds, batch_size=3000, shuffle=True)

for epoch in range(10):
  for i, (bx, by) in enumerate(loader):
    print('Epoch: %d, Iter: %d, Mini-batch Size: %d' % (epoch, i, bx.shape[0]))


num samples: 20000
Epoch: 0, Iter: 0, Mini-batch Size: 3000
Epoch: 0, Iter: 1, Mini-batch Size: 3000
Epoch: 0, Iter: 2, Mini-batch Size: 3000
Epoch: 0, Iter: 3, Mini-batch Size: 3000
Epoch: 0, Iter: 4, Mini-batch Size: 3000
Epoch: 0, Iter: 5, Mini-batch Size: 3000
Epoch: 0, Iter: 6, Mini-batch Size: 2000
Epoch: 1, Iter: 0, Mini-batch Size: 3000
Epoch: 1, Iter: 1, Mini-batch Size: 3000
Epoch: 1, Iter: 2, Mini-batch Size: 3000
Epoch: 1, Iter: 3, Mini-batch Size: 3000
Epoch: 1, Iter: 4, Mini-batch Size: 3000
Epoch: 1, Iter: 5, Mini-batch Size: 3000
Epoch: 1, Iter: 6, Mini-batch Size: 2000
Epoch: 2, Iter: 0, Mini-batch Size: 3000
Epoch: 2, Iter: 1, Mini-batch Size: 3000
Epoch: 2, Iter: 2, Mini-batch Size: 3000
Epoch: 2, Iter: 3, Mini-batch Size: 3000
Epoch: 2, Iter: 4, Mini-batch Size: 3000
Epoch: 2, Iter: 5, Mini-batch Size: 3000
Epoch: 2, Iter: 6, Mini-batch Size: 2000
Epoch: 3, Iter: 0, Mini-batch Size: 3000
Epoch: 3, Iter: 1, Mini-batch Size: 3000
Epoch: 3, Iter: 2, Mini-batch Size: 30

In [ ]:
import torch.nn as nn

class MyModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Linear(784, 128),
        nn.Sigmoid(),
        nn.Linear(128, 64),
        nn.Sigmoid(),
        nn.Linear(64, 32),
        nn.Sigmoid(),
        nn.Linear(32, 10)
    )

  def forward(self, x): #tensor(b, 784)
    return self.layers(x) #tensor(b, 10)

model = MyModel()
tmp = torch.rand(100, 784)
output = model(tmp)
print(output.shape)

torch.Size([100, 10])


In [ ]:
#training
import torch.optim as optim

model = MyModel()
opt = optim.SGD(model.parameters(), lr=0.01)
cost_func = nn.CrossEntropyLoss()

ds = SmallMNIST(x, y)
loader = DataLoader(ds, batch_size=200, shuffle=True)

num_epoch = 100
for epoch in range(num_epoch):
  for i, (bx, by) in enumerate(loader):
    output = model(bx)
    cost = cost_func(output, by)

    if i %100 == 0:
      print('Epoch: %d, Iter: %d, Cost: %.4f' % (epoch, i, cost.item()))

    cost.backward()
    opt.step()
    opt.zero_grad()


Epoch: 0, Iter: 0, Cost: 2.3296
Epoch: 1, Iter: 0, Cost: 2.2985
Epoch: 2, Iter: 0, Cost: 2.3120
Epoch: 3, Iter: 0, Cost: 2.3000
Epoch: 4, Iter: 0, Cost: 2.2924
Epoch: 5, Iter: 0, Cost: 2.2948
Epoch: 6, Iter: 0, Cost: 2.2875
Epoch: 7, Iter: 0, Cost: 2.2933
Epoch: 8, Iter: 0, Cost: 2.2927
Epoch: 9, Iter: 0, Cost: 2.2957
Epoch: 10, Iter: 0, Cost: 2.2868
Epoch: 11, Iter: 0, Cost: 2.2910
Epoch: 12, Iter: 0, Cost: 2.2962
Epoch: 13, Iter: 0, Cost: 2.2930
Epoch: 14, Iter: 0, Cost: 2.2848
Epoch: 15, Iter: 0, Cost: 2.2803
Epoch: 16, Iter: 0, Cost: 2.2830
Epoch: 17, Iter: 0, Cost: 2.2795
Epoch: 18, Iter: 0, Cost: 2.2774
Epoch: 19, Iter: 0, Cost: 2.2786
Epoch: 20, Iter: 0, Cost: 2.2707
Epoch: 21, Iter: 0, Cost: 2.2802
Epoch: 22, Iter: 0, Cost: 2.2675
Epoch: 23, Iter: 0, Cost: 2.2774
Epoch: 24, Iter: 0, Cost: 2.2662
Epoch: 25, Iter: 0, Cost: 2.2592
Epoch: 26, Iter: 0, Cost: 2.2670
Epoch: 27, Iter: 0, Cost: 2.2604
Epoch: 28, Iter: 0, Cost: 2.2605
Epoch: 29, Iter: 0, Cost: 2.2641
Epoch: 30, Iter: 0, 

In [ ]:
#evaluating on training set
num_correct = 0
for bx, by in loader:
  output = model(bx)
  predict = torch.argmax(output, dim=1)
  num_correct += (predict == by).sum().item()

print('Accuracy: %.2f' % (num_correct / len(ds)))


Accuracy: 0.79


In [ ]:
#evaluating on test set

data_test = pd.read_csv('/content/sample_data/mnist_test.csv', header=None).to_numpy()

x_test = data_test[:, 1:]
y_test = data_test[:, 0]

ds_test = SmallMNIST(x_test, y_test)
loader_test = DataLoader(ds_test, batch_size=200, shuffle=False)


In [ ]:
num_correct = 0
for bx, by in loader_test:
  output = model(bx)
  predict = torch.argmax(output, dim=1)
  num_correct += (predict == by).sum().item()

print('Accuracy on Test: %.2f' % (num_correct / len(ds)))

Accuracy on Test: 0.39
